In [13]:
import os
import re
import pandas as pd
import matplotlib.pyplot as plt

def remove_ext(name:str):
    name, _ = os.path.splitext(name)
    return name

data_dict = {} 
subdirs = [s for s in os.listdir(".") if os.path.isdir(os.path.join(".",s)) and s != "figures"]
for model_path in subdirs:
    data_dict[model_path] = {}
    for dataset in ["LASTFM", "REDDIT", "WIKI", "MOOC"]:
        if model_path == "TGAT" and dataset in ["LASTFM", "MOOC"]:
            continue
        data_dict[model_path][dataset] = {"bs":[],"auc":[],"ap":[],"time":[]}
        file_names = [f for f in os.listdir(model_path) if f.startswith(dataset)]
        for file_name in file_names:
            # print(file_name)
            with open(os.path.join(model_path,file_name)) as f:
                lines = f.readlines()
                test_data = lines[-1]
                time_data = lines[-5]
                numbers = [float(num) for num in re.findall(r'\d+\.\d+', test_data)]
                total_time = float(re.findall(r'\d+\.\d+', test_data)[0])
                # print(f"total_time: {total_time}")
                if len(numbers) != 2:
                    continue
                # print(numbers)
                bs = int(remove_ext(file_name).split("_")[1])
                
                data_dict[model_path][dataset]["time"].append(total_time)
                data_dict[model_path][dataset]["bs"].append(bs)
                data_dict[model_path][dataset]["ap"].append(numbers[0])
                data_dict[model_path][dataset]["auc"].append(numbers[1])
                # print(f"bs: {bs}, ap: {numbers[0]}, auc: {numbers[1]}")
                # exit()
                
                
df_list = []
for model, datasets_data in data_dict.items():
    for dataset, dataset_data in datasets_data.items():
        #print(model,dataset)
        df = pd.DataFrame(dataset_data)
        result = df.groupby('bs').mean().reset_index()
        name_column = result['bs'].apply(lambda x: f"{model}_{dataset}_{x}")
        result.insert(0,"name",name_column)
        df_list.append(result)
        # print(result)        
        # print(df_all)
df_all = pd.concat(df_list,ignore_index=True)
df_all = df_all.round(6)
df_all.to_csv("result.csv")
        # plt.figure(figsize=(10, 6))
        # plt.plot(result['bs'], result['auc'], marker='o', label='AUC', color='blue')
        # plt.plot(result['bs'], result['ap'], marker='o', label='AP', color='orange')

        # 添加标题和标签
        # plt.title(f'{model} {dataset}')
        # plt.xlabel('BS')
        # plt.ylabel('AUC/AP')
        # plt.legend()
        # plt.grid()
        # plt.savefig(f"figures/{model}_{dataset}.png")
        # 显示图形
        # plt.show()
        